In [4]:
import networkx as nx
import graph_text_encoder

In [5]:
def create_custom_name_dict():
    """Create a custom name dictionary for a car market graph."""
    return {
        0: 'Toyota',
        1: 'Ford',
        2: 'Tesla',
        3: 'Camry',
        4: 'Mustang',
        5: 'Model S',
        6: 'Dealership A',
        7: 'Dealership B'
    }
    
name_dict = create_custom_name_dict()  
G = nx.Graph()
G.add_nodes_from(range(8))  
# Adding edges (manufacturer to car models, models to dealerships)
G.add_edges_from([(0, 3), (1, 4), (2, 5)])  # Manufacturers to models
G.add_edges_from([(3, 6), (4, 6), (5, 7)])  # Models to dealerships

In [6]:
#Based on the paper, friendship encoder performs the best for zero-shot 
encoded_graph = graph_text_encoder.friendship_encoder(G, name_dict)
print(encoded_graph)

G describes a friendship graph among nodes Toyota, Ford, Tesla, Camry, Mustang, Model S, Dealership A, and Dealership B.
We have the following edges in G:
Toyota and Camry are friends.
Ford and Mustang are friends.
Tesla and Model S are friends.
Camry and Dealership A are friends.
Mustang and Dealership A are friends.
Model S and Dealership B are friends.



In [7]:
from langchain.chains import LLMChain
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
import os
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
prompt_template = """
{context}

Question: {question}
Answer:"""
prompt = PromptTemplate(input_variables=["context", "question"], template=prompt_template)
llm_chain=LLMChain(llm=OpenAI(api_key=api_key), prompt=prompt)

In [16]:
def ask_car_market_question(question):
    prompt_variables = {
        "context": encoded_graph,
        "question": question
    }
    answer = llm_chain.invoke(prompt_variables)
    return answer['text']

In [17]:
question = "Which cars are sold at Dealership B?" #From paper, we know friendship encoding should performs well on edge existence
answer = ask_car_market_question(question)
print("Answer:", answer)

Answer:  Model S


Write a question for cycle check, based on the paper, friendship encoding should performs well on cycle check under the zero-shot

In [18]:
question = "Considering all connections between manufacturers, their cars, and dealerships, can you identify a route that forms a closed loop starting and ending at the same point without visiting any node twice?"
answer = ask_car_market_question(question)
print("Answer:", answer)

Answer:  Yes, there is a route that forms a closed loop: Toyota - Camry - Dealership A - Mustang - Ford - Dealership A - Model S - Tesla - Dealership B - Model S - Dealership A - Camry - Toyota. This route starts and ends at Toyota and does not visit any node twice.
